In [1]:
import pandas
from pandas import *
import numpy
from datetime import datetime



if False:
    df_train = pandas.read_csv("../input/train.csv")
    df_test = pandas.read_csv("../input/test.csv")
else:
    # Get the training data
    df_train = pandas.read_csv("data/train.csv.gz")
    # Only small sample for now
    #
    df_train = df_train.sample(100000)
    # Reading test data set
    df_test = pandas.read_csv("data/test.csv.gz")


print("Starting with {0}".format(df_train.count()))
display(df_train.sample(10))
df_train.describe()

Starting with date     100000
store    100000
item     100000
sales    100000
dtype: int64


,date,store,item,sales
443361,2017-01-09,3,25,56
320761,2016-04-26,6,18,71
323603,2014-02-06,8,18,77
33253,2014-01-21,9,2,41
7310,2013-01-07,5,1,6
37860,2016-09-02,1,3,39
284916,2013-03-02,7,16,8
56681,2013-03-17,2,4,20
508367,2015-01-10,9,28,67
888825,2016-10-21,7,49,26


,store,item,sales
count,100000.000000,100000.000000,100000.000000
mean,5.516380,25.532660,52.213510
std,2.874661,14.402688,28.743739
min,1.000000,1.000000,1.000000
25%,3.000000,13.000000,30.000000
50%,6.000000,26.000000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,208.000000


In [2]:
# Adding various information to the dataframe for later 
# DayOfWeek
# DayOfMonth
# Month
# Year
# CalendarWeek

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

# Add all variables that we would like to this model

# Extracting some variable from date
def date_info(date):
    datetime_object = datetime.strptime(date, "%Y-%m-%d")
    
    week_day = datetime_object.weekday()
    cw = datetime_object.isocalendar()[1]
    
    day = datetime_object.day
    month = datetime_object.month
    year = datetime_object.year
    
    days = (datetime_object - datetime(2010,1,1)).days
    
    return (week_day, cw, day, days, month, year)

def boolean_value(value):
    global current_value
    return True


current_value = 0
def extrapolate(df):
    df['week_day'], df['cw'], df['day'], df['days'], df['month'], df['year'] = zip(*df["date"].map(date_info))

    # Convert date as proper time as well
    df['time'] = df['date'].astype('datetime64[ns]')
    
    # Month should be a category as well
    df['week_day'] = df['week_day'].astype('int32')
    df['month'] = df['month'].astype('int32')

    # Set type of store and item to category
    df['item'] = df['item'].astype('object')
    df['store'] = df['store'].astype('object')
    
    for i in range(1, 11):
        df['s_{0}'.format(i)] = (df['store'] == i)
        
    for i in range(1, 51):
        df['i_{0}'.format(i)] = (df['item'] == i)
    
    # Categorized store and item
    #enc = OneHotEncoder()
    #enc.fit(df['store'])  
    #OneHotEncoder(categorical_features='all', dtype='numpy.float64', handle_unknown='error', n_values='auto', sparse=True)
    #print(enc.transform(df['store']).toarray())
    
    #store_le = preprocessing.LabelEncoder()
    #store_le.fit(df['store'])
    #print(list(store_le.classes_))
    
    #store_le.transform(df['store']) 
    return df

    

# Add those information to df_train and df_test
df_train = extrapolate(df_train)
df_test = extrapolate(df_test)

# Understand how scale change depending on dow
df_dow = df_train
display(df_train.sample(10))


,date,store,item,sales,week_day,cw,day,days,month,year,...,i_41,i_42,i_43,i_44,i_45,i_46,i_47,i_48,i_49,i_50
474331,2016-10-29,10,26,51,5,43,29,2493,10,2016,...,False,False,False,False,False,False,False,False,False,False
738790,2015-12-23,5,41,13,2,52,23,2182,12,2015,...,True,False,False,False,False,False,False,False,False,False
222744,2017-12-04,2,13,66,0,49,4,2894,12,2017,...,False,False,False,False,False,False,False,False,False,False
409346,2013-11-19,5,23,30,1,47,19,1418,11,2013,...,False,False,False,False,False,False,False,False,False,False
844538,2015-07-16,3,47,30,3,29,16,2022,7,2015,...,False,False,False,False,False,False,True,False,False,False
559493,2015-01-08,7,31,22,3,2,8,1833,1,2015,...,False,False,False,False,False,False,False,False,False,False
315249,2016-03-23,3,18,77,2,12,23,2273,3,2016,...,False,False,False,False,False,False,False,False,False,False
667269,2015-02-19,6,37,18,3,8,19,1875,2,2015,...,False,False,False,False,False,False,False,False,False,False
53755,2015-03-13,10,3,39,4,11,13,1897,3,2015,...,False,False,False,False,False,False,False,False,False,False
670629,2014-05-03,8,37,41,5,18,3,1583,5,2014,...,False,False,False,False,False,False,False,False,False,False


In [3]:
# Generate our training/validation datasets
from sklearn import model_selection

result_cols = ['sales']
# Removing input_cols = ['store', 'item',
input_cols = [
              'week_day',
              'days',
              'cw',
              'day', 
              'days','month', 'year']

for i in range(1, 11):
    input_cols.append('s_{0}'.format(i))

for i in range(1, 51):
    input_cols.append('i_{0}'.format(i))

def get_values(df, cols=[]):
    final_df = df
    
    print(final_df.columns.values)
    # Remove all columns that are not inside the list
    for column in final_df.columns.values:
        if column not in cols:
            final_df = final_df.drop(columns=[column])
    return final_df.values


X = get_values(df_train, input_cols)
Y = get_values(df_train, result_cols).ravel()
X_test = get_values(df_test, input_cols)

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Create validation + training set

['date' 'store' 'item' 'sales' 'week_day' 'cw' 'day' 'days' 'month' 'year'
 'time' 's_1' 's_2' 's_3' 's_4' 's_5' 's_6' 's_7' 's_8' 's_9' 's_10' 'i_1'
 'i_2' 'i_3' 'i_4' 'i_5' 'i_6' 'i_7' 'i_8' 'i_9' 'i_10' 'i_11' 'i_12'
 'i_13' 'i_14' 'i_15' 'i_16' 'i_17' 'i_18' 'i_19' 'i_20' 'i_21' 'i_22'
 'i_23' 'i_24' 'i_25' 'i_26' 'i_27' 'i_28' 'i_29' 'i_30' 'i_31' 'i_32'
 'i_33' 'i_34' 'i_35' 'i_36' 'i_37' 'i_38' 'i_39' 'i_40' 'i_41' 'i_42'
 'i_43' 'i_44' 'i_45' 'i_46' 'i_47' 'i_48' 'i_49' 'i_50']
['date' 'store' 'item' 'sales' 'week_day' 'cw' 'day' 'days' 'month' 'year'
 'time' 's_1' 's_2' 's_3' 's_4' 's_5' 's_6' 's_7' 's_8' 's_9' 's_10' 'i_1'
 'i_2' 'i_3' 'i_4' 'i_5' 'i_6' 'i_7' 'i_8' 'i_9' 'i_10' 'i_11' 'i_12'
 'i_13' 'i_14' 'i_15' 'i_16' 'i_17' 'i_18' 'i_19' 'i_20' 'i_21' 'i_22'
 'i_23' 'i_24' 'i_25' 'i_26' 'i_27' 'i_28' 'i_29' 'i_30' 'i_31' 'i_32'
 'i_33' 'i_34' 'i_35' 'i_36' 'i_37' 'i_38' 'i_39' 'i_40' 'i_41' 'i_42'
 'i_43' 'i_44' 'i_45' 'i_46' 'i_47' 'i_48' 'i_49' 'i_50']
['id' 'date' 'stor

In [4]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(X_train)  
OneHotEncoder(categorical_features='all', handle_unknown='error', n_values='auto', sparse=True)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [5]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor



# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

# Spot Check Algorithms
models = []
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('TREEREG', DecisionTreeRegressor()))

#models.append(('NB', GaussianNB()))
models.append(('NEURAL', MLPClassifier()))
#models.append(('LR', LogisticRegression()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []

import sklearn
print(sklearn.__version__)


# Simple function to represent the result
def mean_absolute_percentage_error(Y_validation, Y_validation_predict):
    result = 0
    for i in range(0, len(Y_validation)):
        result += (abs(Y_validation[i] - Y_validation_predict[i]))/(abs(Y_validation[i])+abs(Y_validation_predict[i]))
    return result / len(Y_validation) * 200
print("Starting all models")
#Y_train=Y_train.astype('int')

max_score = 10000
for name, model in models:
    print("Executing for model {0}".format(name))
    #kfold = model_selection.KFold(n_splits=10, random_state=seed)
    #cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    #results.append(cv_results)
    #names.append(name)
    #msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    #print(msg)
    
    # Writing ht
    
    # Training the model
    model.fit(X_train, Y_train)
    
    Y_validation_predict = model.predict(X_validation)
    score = mean_absolute_percentage_error(Y_validation, Y_validation_predict)
    print("Model {0} got score of {1}".format(name, score))
    
    Y_test = model.predict(X_test)
    print("Result should be {0}".format(Y_test))

    # Create new result dataframe:
    result_df = df_test['id']
    df_test['sales'] = Y_test
    df_test.head(5)
    df_result = df_test[['id', 'sales']]

    # Write the dataframe as output
    df_result.head(5)

    df_result.to_csv("output_{0}.csv".format(name), index=False)
    if max_score > score:
        max_score = score
        print("Choosing {0} as best result".format(name))
        df_result.to_csv("output_best.csv".format(name), index=False)
print("Final result should be around {0}".format(max_score))


0.19.2
Starting all models
Executing for model NEURAL
Model NEURAL got score of 15.32081386184935
Result should be [10 13 13 ..., 72 88 88]
Choosing NEURAL as best result
Final result should be around 15.32081386184935
